In [ ]:
try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [ ]:
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [ ]:
# import local files
import geetools
import main
import indices

### General Workflow

1. Import the area of interest (soum shapefile is first uploaded to GEE asset)
2. Import mask areas (Soum Seasonal Range Areas)
3. Filter data
    - Date
    - Cloud Coverage
    - Best pixel
4. Apply functions
    - Landsat sensors harmonization
    - Cloudless pixels
    - Vegetation indices
    - Custom masks
5. Download datasets per month and year per soum 


In [ ]:
# Import area of interest
# bags = ee.FeatureCollection('users/ta346/mng_grassland/mng_nso_bag')
soums = ee.FeatureCollection('users/ta346/mng_boundary_cleaned/mn_soum_cln')

In [ ]:
#import seasonal
# winter - 1, summer - 2, pasture-not-used - 3
pasture = ee.Image("users/ta346/pasture_delineation/pas_raster_new")
# import landcover (20m) tiff image from asset in Google Earth Engine
# [10, 20, 60, 80, 100]
# search what files are in the GEE asset folder geemap.ee_search()
lc2020 = ee.Image('users/ta346/mng_landcover_30m/2020LC30').clip(soums).select('b1')
lc2010 = ee.Image('users/ta346/mng_landcover_30m/2010LC30').clip(soums).select('b1')
lc2000 = ee.Image('users/ta346/mng_landcover_30m/2000LC30').clip(soums).select('b1')

In [ ]:
# vegetation index interests 
index_need = ['ndvi', 'evi', 'savi', 'msavi', 'nirv']
# create a list of names for mask
download = ['sgr', 'wgr', 'gr']

### 1. Landsat Collection 

In [ ]:
# cloud free landsat collection
landsat_collection = main.get_landsat_collection(dateIni='1984-01-01', # initial date
                                                        dateEnd='2021-12-31', # end date
                                                        box=soums, # area of interest
                                                        perc_cover=50, # only images where more than 50% of pixels are cloud free
                                                        sensor=["LC08", "LE07", "LT05"], # search for all available sensors
                                                        harmonization=True) # ETM and ETM+ to OLI

In [ ]:
# Compute vegetation indices on cloud free landsat collection
landsat_collection = (landsat_collection.map(indices.ndvi(nir= "SR_B4", red = "SR_B3", bandname = "ndvi"))
                                   .map(indices.evi(nir = "SR_B4", red = "SR_B3", blue = "SR_B1", G = 2.5, C1 = 6, C2 = 7.5, L=1, bandname='evi'))
                                   .map(indices.savi(nir = "SR_B4", red = "SR_B3", L = 0.5, G = 1.5, bandname="savi"))
                                   .map(indices.msavi(nir = "SR_B4", red = "SR_B3", G = 2, H = 8, L = 1, bandname="msavi"))
                                   .map(indices.nirv(nir = "SR_B4", red = "SR_B3", bandname="nirv"))
                                   .map(indices.ndwi(nir = "SR_B4", swir = "SR_B5", bandname="ndwi")))

In [ ]:
# save each resulting CSV's to Google Drive Folder 
# this exports landsat collection per year on three different sceneries:
    # a. winter grazing ranges (wgr)
    # b. summer grazing ranges (sgr)
    # c. both grazing ranges (gr)
for i in download:
    other_mask = pasture
    if i == 'wgr':
        other_mask_parameter = [2,3]
    elif i == 'sgr':
        other_mask_parameter = [1,3]
    elif i == 'gr':
        other_mask_parameter = [0]
    file_name = str(i) + "_" + "pasture_vi_soum"
    folder_name = "LANDSAT" + "SOUM_VI_" + str(i).upper()

    comp = main.download_img_col_stats_to_csv_yearly(landsat_collection, 
                                                    bandnames = index_need, 
                                                    box = soums, 
                                                    reducerAll = True,
                                                    feat_name='asid', 
                                                    scale = 30,
                                                    tileScale = 1,  
                                                    crs = "EPSG:4326",
                                                    other_mask = other_mask, 
                                                    other_mask_parameter=other_mask_parameter, 
                                                    file_name = file_name,
                                                    folder_name=folder_name)

### 2. MODIS43A

In [ ]:
# modis46A cloudless image collection
modis46a = main.get_modis46a_500_collection(dateIni='2002-02-24', 
                                                dateEnd='2021-12-31', 
                                                box = soums, 
                                                quality_mask=True) 

In [ ]:
# Compute vegetation indices on cloud free landsat collection
modis46a = (modis46a.map(indices.ndvi(nir = "Nadir_Reflectance_Band2", red = 'Nadir_Reflectance_Band1', bandname='ndvi'))
                .map(indices.evi(nir = 'Nadir_Reflectance_Band2', red = 'Nadir_Reflectance_Band1', blue = 'Nadir_Reflectance_Band3', bandname='evi'))
                .map(indices.savi(nir = 'Nadir_Reflectance_Band2', red = 'Nadir_Reflectance_Band1', bandname='savi'))
                .map(indices.msavi(nir = 'Nadir_Reflectance_Band2', red = 'Nadir_Reflectance_Band1', bandname='msavi'))
                .map(indices.nirv(nir = 'Nadir_Reflectance_Band2', red = 'Nadir_Reflectance_Band1', bandname='nirv')))

In [ ]:
# save each image collection per year
for i in download:
    other_mask = pasture
    if i == 'wgr':
        other_mask_parameter = [2,3]
    elif i == 'sgr':
        other_mask_parameter = [1,3]
    elif i == 'gr':
        other_mask_parameter = [0]
    file_name = str(i) + "_" + "pasture_vi_soum"
    folder_name = "MODIS43A" + "SOUM_VI_" + str(i).upper()

    comp = main.download_img_col_stats_to_csv_yearly(modis46a, 
                                                    bandnames = index_need, 
                                                    box = soums, 
                                                    reducerAll = True,
                                                    feat_name='asid', 
                                                    scale = 1000,
                                                    tileScale = 1,  
                                                    crs = "EPSG:4326",
                                                    other_mask = other_mask, 
                                                    other_mask_parameter=other_mask_parameter, 
                                                    file_name = file_name,
                                                    folder_name=folder_name)

### 3. Weather

In [ ]:
# band names
select_band = ["temperature_2m", 
               "snow_cover", 
               "snow_density", 
               "snow_depth", 
               "snow_depth_water_equivalent", 
               "snowfall", 
               "snowmelt", 
               "u_component_of_wind_10m", 
               "v_component_of_wind_10m", 
               "total_precipitation"]

In [ ]:
# get era5 collection
era5 = main.get_era5_collection('1984-01-01', '2021-12-31', soums, select_band)

In [ ]:
# save each image collection as CSV's per month to Google Drive Folder 
for i in download:
    other_mask = pasture
    if i == 'wgr':
        other_mask_parameter = [2,3]
    elif i == 'sgr':
        other_mask_parameter = [1,3]
    elif i == 'gr':
        other_mask_parameter = [0]
    file_name = str(i) + "_" + "weather_soum"
    folder_name = "ERA5_SOUM" + str(i).upper()

    # download each image per year as csv file over area of interest at soum level
    comp = main.download_img_col_to_csv_monthly(era5, 
                                                    startYear = 1984,
                                                    endYear = 2021,
                                                    startMonth = 1,
                                                    endMonth = 12,
                                                    bandnames = select_band, 
                                                    box = soums, 
                                                    reducerAll = True,
                                                    feat_name='asid', 
                                                    scale = 9000,
                                                    tileScale = 1,
                                                    other_mask = other_mask,
                                                    other_mask_parameter = other_mask_parameter,  
                                                    crs = "EPSG:4326", 
                                                    file_name = file_name,
                                                    folder_name=folder_name)

Download Summer NDVI as Raster Images

In [ ]:
def get_median_summer_ndvi(startYear, endYear, image_collection):
    startYear = startYear
    endYear = endYear

    loopsteps = ee.List.sequence(startYear, endYear,1)

    def get_median(year):
        iniDate = ee.Date.fromYMD(year, 6, 1)
        endDate = iniDate.advance(3, "month")

        date = iniDate.format("YYYY")
        img_col = (image_collection.filter(ee.Filter.date(iniDate, endDate))
                                                .select('ndvi')
                                                .reduce(ee.Reducer.median())
                                                .clip(soums)
                                                .set('system:index', date)
                                                .set('system:time_start', iniDate.millis())
                                                .set('system:time_end', endDate.millis()))
        return img_col
    
    median_composites = ee.ImageCollection.fromImages(loopsteps.map(get_median))

    return median_composites

In [ ]:
ls_median_summer_ndvi = get_median_summer_ndvi(1985, 2021, landsat_collection)
modis_median_summer_ndvi = get_median_summer_ndvi(2002, 2021, modis46a)

In [ ]:
stack_ls = ls_median_summer_ndvi.toBands()
stack_modis = modis_median_summer_ndvi.toBands()

In [ ]:
ls_median_summer_ndvi.getInfo()

In [ ]:
colorizedVis = {
  "min": 0.0,
  "max": 1.0,
  "palette": [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
}

ls_median_2015 = ls_median_summer_ndvi.filter(ee.Filter.date('2015-01-01', '2015-12-31')).first()
ls_median_2016 = ls_median_summer_ndvi.filter(ee.Filter.date('2016-01-01', '2016-12-31')).first()
modis_median_2015 = modis_median_summer_ndvi.filter(ee.Filter.date('2015-01-01', '2015-12-31')).first()
modis_median_2016 = modis_median_summer_ndvi.filter(ee.Filter.date('2016-01-01', '2016-12-31')).first()

left_layer = geemap.ee_tile_layer(stack_ls.select("2016_ndvi_median"), colorizedVis, "Landsat 2015 (30m): Summer Median NDVI")
right_layer = geemap.ee_tile_layer(stack_modis.select("2016_ndvi_median"), colorizedVis, "MODIS 2015 (250m): Summer Median NDVI")

Map = geemap.Map()
Map.centerObject(soums)
Map.addLayer(soums, {}, "soum boundary")
Map.addLayer(stack_ls.select("2021_ndvi_median"), colorizedVis, "Landsat 2016 (30m): SUMMER NDVI")
Map.addLayer(stack_modis.select("2021_ndvi_median"), colorizedVis, "Modis 2016 (30m): SUMMER NDVI")
Map

In [ ]:
era5 = era5.select('temperature_2m', 'total_precipitation')

In [13]:
era5.toBands()

In [ ]:
# Export the image, specifying the CRS, transform, and region.
task_config = {
    'fileNamePrefix': 'era5_stack_weather',
    'crs': 'EPSG:4326',
    'scale': 9000,
    'maxPixels': 10e12,
    'fileFormat': 'GeoTIFF',
    'skipEmptyTiles': True,
    'region': soums.geometry(),
    'folder': 'LS_RASTER'
    }

task = ee.batch.Export.image.toDrive(stack_ls, str('era5_stack'), **task_config)

In [ ]:
task.start()

In [ ]:
task.status()